# ARIMA and SARIMA
This notebook develops the arima model.

## Imports:

In [ ]:
import os
import pandas as pd
from pathlib import Path
import pmdarima as pm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import root_mean_squared_error
import pickle
import sys
sys.path.append(os.path.join('..', 'Helper'))
sys.path.append(os.path.join('..', '..', 'Evaluation', 'Helper'))
from dataPreprocessing import make_stationary, granger_causes, rank_features_ccf, get_untransformed_exog
from evaluation_helpers import display_results

## Data:

In [ ]:
cwd=Path.cwd()
data= pd.read_csv(cwd.parent.parent / 'Data' /'Train'/'train1990s.csv',parse_dates=[0],date_format='%m%Y',index_col=0)

In [ ]:
unmodifiedDf = get_untransformed_exog(data)

split= round(unmodifiedDf.shape[0]*0.9)
train= unmodifiedDf.iloc[:split+1,:]
valid= unmodifiedDf.iloc[split+1:,:]

# Preprocessing
ARIMA requires the data to be stationary, however AutoArima kindly handles it for you!

# ARIMA

In [ ]:
#Make seasonal False for ARIMA (otherwise its SARIMA)
arima_model= pm.auto_arima(train['fred_PCEPI'],start_p=1, start_q=1,seasonal=False, stepwise=True)
preds=np.array(arima_model.predict(valid.shape[0]))
Y= np.array(valid.iloc[:,0])

#could instead use evaluate_model here, consider changing
print(f'RMSE: {root_mean_squared_error(Y,preds)}')
display_results(Y, preds, list(valid.index), 'ARIMA', print_dates=10)

# ARIMAX

In [ ]:
#Make seasonal False for ARIMAX (otherwise its SARIMA)
arimax_model= pm.auto_arima(y=train['fred_PCEPI'],X=train.drop(['fred_PCEPI'],axis=1),start_p=1, start_q=1,seasonal=False, stepwise=True)
preds=np.array(arimax_model.predict(valid.shape[0],valid.drop(['fred_PCEPI'],axis=1)))
Y= np.array(valid['fred_PCEPI'])

print(f'RMSE: {root_mean_squared_error(Y,preds)}')
display_results(Y, preds, list(valid.index), 'ARIMAX', print_dates=10)

## SARIMA

In [ ]:
#change seasonal to True and m=12 for SARIMA
sarima_model= pm.auto_arima(train['fred_PCEPI'],start_p=1, start_q=1,seasonal=True,m=12, stepwise=True)
preds=np.array(sarima_model.predict(valid['fred_PCEPI'].shape[0]))
Y= np.array(valid.iloc[:,0])
print(f'RMSE: {root_mean_squared_error(Y,preds)}')
display_results(Y, preds, list(valid.index), 'SARIMA', print_dates=10)

## SARIMAX

In [ ]:
#change seasonal to True and m=12 for SARIMA
sarimax_model= pm.auto_arima(y=train['fred_PCEPI'],X=train.drop(['fred_PCEPI'],axis=1),start_p=1, start_q=1,m=12,seasonal=True, stepwise=True)
preds=np.array(sarimax_model.predict(valid.shape[0],valid.drop(['fred_PCEPI'],axis=1)))
Y= np.array(valid['fred_PCEPI'])

print(f'RMSE: {root_mean_squared_error(Y,preds)}')
display_results(Y, preds, list(valid.index), 'SARIMAX', print_dates=10)

As shown ARIMAX and SARIMAX are sensitive to large numbers of exogenous variables, therefore some method is required to rank the best features:
mutual_info_regression
cross correlation
Variance Inflation Factor (VIF) (Remove variables with VIF > 5–10 to avoid unstable coefficient estimates)

## Feature Selection:
ARIMAX and SARIMAX are sensitive to the amount of feratures used and should

In [ ]:
def remove_noncaused(df:pd.DataFrame, targetCol:str):

    '''
    This function removes features which are not granger caused.

    Parameters:
    -----------
    df: panda dataframe with all exogenous variables and target.

    targetCol: name of target variable in df.

    Returns:
    --------
    Returns a datframe with all features being granger caused.
    '''

    df_selected= df.copy()
    
    dropCols=[]
    # Loop over columns:
    for col in df_selected.columns.drop(targetCol):

        stationary= make_stationary(df_selected,col,targetCol)# makes stime series stationary

        #remove columns which do not achieve stationarity
        if stationary is np.nan:
            dropCols.append(col)
            
        # Removes columnns which do not granger cause (sanity check)
        elif not granger_causes(df_selected,col,targetCol):
            dropCols.append(col)
    
    return df_selected.drop(dropCols, axis=1)

def SARIMAX_grid_search(ranked_df,targetCol, valid=0.1, top_n=20, verbose=False):
    '''
    Performs a grid search on the optimal number of exogenous variables to use.

    Parameters:
    -----------
    ranked_df: pandas dataframe where the columns are ordered in descending order of predictive power (ccf in this case).

    targetCol: name of target column in ranked_df.

    valid: the validation size (defaults to 10% of training data).

    top_n: gridsearch up top_n exogenous variables in total.

    Returns:
    --------
    Tuple, where the 1st element is the optimal RMSE value,
    the 2nd element is the best number of features to use. 
    '''
    # Separate the target and exogenous variables:
    target= ranked_df[targetCol]
    df_exog= ranked_df.drop(targetCol, axis=1)

    # Split into training data and validation data
    split= round(ranked_df.shape[0]*(1. - valid))
    train_exog= df_exog.iloc[:split+1,:]
    valid_exog= df_exog.iloc[split+1:,:]

    train_target= target.iloc[:split+1]
    valid_target= target.iloc[split+1:]

    # Values used to find the optimal solution
    best_score=999
    best_n=-1

    # loop over possible n's:
    for i in range(1,top_n+1):
        if verbose: print(f'Training with {i+1} exogenous variables...')
        #train model:
        sarimax_model= pm.auto_arima(y=train_target,X=train_exog.iloc[:,:i],start_p=1, start_q=1,m=12,seasonal=True, stepwise=True)

        #Infer on the validation data:
        preds=np.array(sarimax_model.predict(valid_exog.shape[0],valid_exog.iloc[:,:i]))
        
        # Calculate RMSE score
        Y= np.array(valid_target)
        score=root_mean_squared_error(Y,preds)

        # Update if a new minimum is found:
        if score < best_score:
            if verbose: print(f'New best score {score:.6f} found.')
            best_score=score
            best_n=i


    print(f"Best score: {best_score}")
    print(f"Best number of features: {best_n}")

    return best_score, best_n

## Find optimal number of exogenous variables:

In [ ]:
# Remove non-granger-caused features, then find their ranking order by cross-correlation
caused_df = remove_noncaused(unmodifiedDf,'fred_PCEPI')
ranked_columns=rank_features_ccf(caused_df,'fred_PCEPI')

In [ ]:
# Reorder DataFrame of caused features according to rankings
# Removed target from caused_df to ensure has the same columns as provided for ordering, then add target back in at front
ranked_df = caused_df.drop('fred_PCEPI', axis=1)
ranked_df = ranked_df[list(ranked_columns)]
ranked_df.insert(0, 'fred_PCEPI', caused_df['fred_PCEPI'].values)

In [ ]:
#best_score,best_n=SARIMAX_grid_search(ranked_df,'fred_PCEPI', top_n=20,valid=0.2)
best_score,best_n=SARIMAX_grid_search(ranked_df,'fred_PCEPI', top_n=1,valid=0.2)

10% validation size:

Best score: 0.39031823257263404

Best number of features: 11

20% validation size:

Best score: 0.20367743319713377

Best number of features: 9

Using a validaion size of 10% overfitted, hence 20% generalized better.

## Train final model:

trains the model using the optimal parameters and saves the model to a pickle file

In [ ]:
target= ranked_df['fred_PCEPI']
df_exog= ranked_df.drop('fred_PCEPI', axis=1)
sarimax_model_optimal_full= pm.auto_arima(y=target,X=df_exog.iloc[:,:best_n],start_p=1, start_q=1,m=12,seasonal=True, stepwise=True)

In [ ]:
with open('SARIMAX.pkl','wb') as f:
    pickle.dump(sarimax_model_optimal_full,f)

# Evaluation:


In [ ]:
# get top 9 column names
used_cols=df_exog.columns[:best_n]
test_data=pd.read_csv(cwd.parent.parent / 'Data' /'Test'/'test1990s.csv',parse_dates=[0],date_format='%m%Y',index_col=0)
test_exog= test_data[used_cols]
test_targets=test_data['fred_PCEPI']
display(test_exog)

In [ ]:
with open('SARIMAX.pkl','rb') as f:
    loaded_model=pickle.load(f)

    preds=np.array(loaded_model.predict(test_exog.shape[0],test_exog))
    Y= np.array(test_targets)

    display(preds.shape, Y.shape)

    print(f'RMSE: {root_mean_squared_error(Y,preds)}')
    display_results(Y, preds, list(test_exog.index), 'SARIMAX', print_dates=10)
    np.save('../../Predictions/SARIMAX.npy',preds)